## Model problem
Let $\Omega$ be a bounded polygonal domain in $\mathbb{R}^2$. The Stokes equations for an incompressible viscous fluid confined in $\Omega$ are
\begin{equation}
    \begin{aligned}
        -\mu\Delta \boldsymbol{u} + \nabla p &= \boldsymbol{f} \quad \text{in } \quad \Omega, \\ 
        \nabla \cdot \boldsymbol{u} &= 0 \quad \text{in } \quad \Omega, \\
        \boldsymbol{u} &= \boldsymbol{0} \quad \text{on } \quad \partial\Omega.         
    \end{aligned}
\end{equation}

- $\boldsymbol{u}$: the fluid velocity
- $p$: the fluid pressure
- $\mu > 0$: the fluid viscosity
- $\boldsymbol{f}$: a body force acting on the fluid

Since $p$ is uniquely defined up to an additive constant, we also assume that $\int_{\Omega} p = 0$. 

## Weak formulation
If we assume that $\boldsymbol{f} \in L^2(\Omega)^2$, a weak solution to the model problem is the pair $(\boldsymbol{u}, p) \in H_0^1(\Omega)^2 \times L_0^2(\Omega)$ satisfying
\begin{equation}
    \begin{aligned}
        \mu(\nabla \boldsymbol{u}, \nabla \boldsymbol{v})_{\Omega} - (\nabla \cdot \boldsymbol{v}, p)_{\Omega} &= (\boldsymbol{f}, \boldsymbol{v})_{\Omega} \quad &&\forall \boldsymbol{v} \in H_0^1(\Omega)^2, \\
        (\nabla \cdot \boldsymbol{u}, q)_{\Omega} &= 0 \quad &&\forall q \in L_0^2(\Omega).
    \end{aligned}
\end{equation}

The space of **divergence-free vector functions** is defined by
\begin{equation}
    \boldsymbol{V} = \{\boldsymbol{v} \in H_0^1(\Omega)^2 : \forall q \in L_0^2(\Omega), (\nabla \cdot \boldsymbol{v}, q)_{\Omega} = 0\}.
\end{equation}

The space $\boldsymbol{V}$ is equipped with the norm $\boldsymbol{v} \mapsto \|\nabla \boldsymbol{v}\|_{L^2(\Omega)^2}$. 

## DG scheme
Let $\mathcal{E}_h$ be a mesh of $\Omega$. For any integer $k \geq 2$, we define the discrete velocity and pressure spaces:
\begin{equation}
    \begin{aligned}
        X_h &= \{\boldsymbol{v} \in L^2(\Omega)^2 : \forall E \in \mathcal{E}_h, \boldsymbol{v} \in (\mathbb{P}_k(E))^2\}, \\
        M_h &= \left\{q \in L_0^2(\Omega) : \forall E \in \mathcal{E}_h, q \in \mathbb{P}_{k - 1}(E)\right\}.
    \end{aligned}
\end{equation}

We introduce the bilinear forms $a_{\epsilon} : X_h \times X_h \to \mathbb{R}$ and $b : X_h \times M_h \to \mathbb{R}$ corresponding to DG discretizations of the diffusive term $-\Delta \boldsymbol{u}$ and the pressure term $\nabla p$, respectively:

\begin{equation}
    \begin{aligned}
        a_{\epsilon}(\boldsymbol{w}, \boldsymbol{v}) 
        &= \sum_{E \in \mathcal{E}_h} \int_E \nabla \boldsymbol{w} : \nabla \boldsymbol{v} + \sum_{e \in \Gamma_h \cup \partial\Omega} \frac{\sigma^0_e}{|e|} \int_e [\boldsymbol{w}] \cdot [\boldsymbol{v}] \\
        &- \sum_{e \in \Gamma_h \cup \partial\Omega} \int_e \{\nabla \boldsymbol{w}\} \mathbf{n}_{e} \cdot [\boldsymbol{v}] + \epsilon \sum_{e \in \Gamma_h \cup \partial\Omega} \int_e \{\nabla \boldsymbol{v}\} \boldsymbol{n}_e \cdot [\boldsymbol{w}], \\
        b(\boldsymbol{v}, q) &= -\sum_{E \in \mathcal{E}_h} \int_E q \nabla \cdot \boldsymbol{v} + \sum_{e \in \Gamma_h \cup \partial\Omega} \int_e \{q\} [\boldsymbol{v}] \cdot \boldsymbol{n}_e.
    \end{aligned}
\end{equation}

With these spaces and bilinear forms, the numerical method is as follows: Find $(\boldsymbol{U}_h, p_h) \in X_h \times M_h $ such that

\begin{equation}
    \begin{aligned}
        \mu a_{\epsilon}(\boldsymbol{U}_h, \boldsymbol{v}) + b(\boldsymbol{v}, p_h) &= (\boldsymbol{f}, \boldsymbol{v})_{\Omega} \quad &&\forall \boldsymbol{v} \in X_h, \\
        b(\boldsymbol{U}_h, q) &= 0 \quad &&\forall q \in M_h.
    \end{aligned}
\end{equation}


In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from math import pi

In [52]:
# Define important parameters
epsilon = -1
sigma = 10 # penalty parameter
order = 2  # k
mh=0.05
mu = 1
f_vec = CoefficientFunction((1+2*pi**3*sin(2*pi*y)*(1-2*cos(2*pi*x)), 2*pi**3*sin(2*pi*x)*(2*cos(2*pi*y)-1)))
# dirichlet_bnd = "left|right|top"
# gD = sin(x) * sin(y)
exact_p = x-1/2
exact_u = CoefficientFunction((pi*sin(pi*x)**2*sin(2*pi*y), -pi*sin(pi*y)**2*sin(2*pi*x)))

In [53]:
mesh = Mesh(unit_square.GenerateMesh(maxh=mh))
# Draw(mesh)

In [54]:
X = VectorL2(mesh, order=order,dgjumps=True, dirichlet=".*")
M = L2(mesh, order=order-1, dgjumps=True)
# M = H1(mesh, order=order-1)
V = X*M


In [55]:
# print(X.ndof)
# print(X.FreeDofs())
# print(M.ndof)
# print(M.FreeDofs())

In [56]:
(u,p), (v,q) = V.TnT()

# Define the jumps and the averages
jump_u = u - u.Other()
jump_v = v - v.Other()
n = specialcf.normal(2)
mean_dudn = 0.5*(grad(u)+grad(u.Other()))*n
mean_dvdn = 0.5*(grad(v)+grad(v.Other()))*n
mean_p = 0.5*(p + p.Other())
mean_q = 0.5*(q + q.Other())
h = specialcf.mesh_size   

In [57]:
Ah = BilinearForm(V)

Ah += InnerProduct(Grad(u), Grad(v))*dx
Ah += (epsilon * mean_dvdn * jump_u- mean_dudn * jump_v)*dx(skeleton=True) \
    + sigma/h* jump_u*jump_v*dx(skeleton=True) 
Ah += (epsilon * grad(v).Trace()*n*u - grad(u).Trace()*n*v)*ds(skeleton=True)  \
    + sigma/h*u*v*ds(skeleton=True)

Ah += - p*div(v)*dx + mean_p*jump_v*n*dx(skeleton=True) + p*v*n*ds(skeleton=True)
Ah += - q*div(u)*dx + mean_q*jump_u*n*dx(skeleton=True) + q*u*n*ds(skeleton=True)
# Ah += - p*div(v)*dx + p*jump_v*n*dx(skeleton=True) + p*v*n*ds(skeleton=True)
# Ah += - q*div(u)*dx + q*jump_u*n*dx(skeleton=True) + q*u*n*ds(skeleton=True)

Ah.Assemble()

In [58]:
Fh = LinearForm(V)
Fh += InnerProduct(f_vec,v)*dx
Fh.Assemble()

In [59]:
gf = GridFunction(V)
gf.vec.data = Ah.mat.Inverse()*Fh.vec

In [60]:
gfu, gfp = gf.components
Draw(gfu, mesh);
Draw(gfp, mesh);
p_avg = Integrate(gfp, mesh) / Integrate(1, mesh)
gfp -= p_avg  # 使压力平均值为0
Draw(gfp, mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [61]:
error_u = sqrt(Integrate((gfu - exact_u)**2, mesh))
print(error_u)
error_p = sqrt(Integrate((gfp - exact_p)**2, mesh))
print(error_p)

0.00022690297939644675
0.016694247208684582
